# **K-Pop가사 생성모델 만들기(미세튜닝)**


---


- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.


---

- **목표**
    - 한국어를 잘하는 경량화 모델(예: KoGPT2 또는 Polyglot-Ko 5.8B 등)을 가져와,  
    - K-pop 가사 데이터셋으로 추가 학습(**Fine-tuning**)시켜
    - **K-Pop가사 생성모델 만들기**
- **데이터셋**
    - **EX3exp / Kpop-lyric-datasets** (GitHub)
    - https://github.com/EX3exp/Kpop-lyric-datasets



---



In [1]:
%%capture
!pip install transformers accelerate tokenizers datasets safetensors



---



## **도메인 특화 파인튜닝 예**

### **예제 : K-pop 가사 스타일 생성기**
- - 데이터 추가해서 미세튜닝하는 작업의 필요성 확인하

In [2]:
# ========================================
# K-pop 가사 스타일 생성기
# (기존 모델을 K-pop 가사로 간단히 파인튜닝하는 예제)
# ========================================

print("="*70)
print("🎵 K-pop 가사 생성기 - 파인튜닝 예제")
print("="*70)

import os, torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, GPT2LMHeadModel  # Trainer/TrainingArguments는 조건부 임포트
)

DO_TRAIN = False  # 🔁 실제 학습하려면 True로 변경

# 1) 데이터
kpop_lyrics = [
    "넌 나의 빛이야 어둠 속에서도 빛나는 별처럼",
    "우리의 꿈을 향해 함께 날아가자 높이 더 높이",
    "사랑해 사랑해 이 마음 전해질까",
    "반짝이는 무대 위 우리는 하나가 돼",
    "Don't stop the music 계속 춤춰봐",
    "You're my star 내 하늘의 유일한 빛",
    "함께라면 두렵지 않아 Let's go",
    "빛나는 우리의 순간 Forever young",
    "손을 잡고 함께 가자 끝까지",
    "Dreams come true 믿어봐 우리의 미래",
]
dataset = Dataset.from_dict({"text": kpop_lyrics})
print(f"✅ 데이터셋 생성 완료: {len(dataset)} 샘플")

# 2) 모델/토크나이저
model_name = "skt/kogpt2-base-v2"
print(f"\n📥 베이스 모델 로드: {model_name}")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# pad/eos 세팅
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id
# (신규 토큰 추가시) model.resize_token_embeddings(len(tokenizer))

# 3) 토큰화
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=128, padding="max_length")
print("\n🔄 데이터 토큰화 중...")
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
print("✅ 토큰화 완료")

# 4) (옵션) 학습
if DO_TRAIN:
    print("\n🎓 파인튜닝 시작")
    from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    training_args = TrainingArguments(
        output_dir="./kpop-gpt2",
        num_train_epochs=1,
        per_device_train_batch_size=2,
        learning_rate=5e-5,
        weight_decay=0.01,
        logging_steps=5,
        save_steps=10,
        save_total_limit=2,
        push_to_hub=False,
        report_to=[],
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator,
    )
    trainer.train()
    trainer.save_model("./kpop-gpt2")

# 5) 생성 데모 (학습 전/후 동일하게 사용 가능)
print("\n" + "="*70)
print("🎤 K-pop 스타일 가사 생성 (베이스/혹은 파인튜닝 모델)")
print("="*70)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

generation_prompts = ["사랑해", "함께", "꿈을"]
for i, prompt in enumerate(generation_prompts, 1):
    print(f"\n【 생성 {i} 】 시작 단어: {prompt}")
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            temperature=0.9,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    print("-"*70)


print("\n💡 실제 파인튜닝:")
print("   1. 더 많은 K-pop 가사 데이터 수집 (1000+ 곡)")
print("   2. 위 TrainingArguments로 학습 실행")
print("   3. 파인튜닝된 모델로 더 나은 결과 생성")
print("   4. Hugging Face Hub에 업로드하여 공유")

🎵 K-pop 가사 생성기 - 파인튜닝 예제
✅ 데이터셋 생성 완료: 10 샘플

📥 베이스 모델 로드: skt/kogpt2-base-v2


config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]


🔄 데이터 토큰화 중...


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

✅ 토큰화 완료

🎤 K-pop 스타일 가사 생성 (베이스/혹은 파인튜닝 모델)

【 생성 1 】 시작 단어: 사랑해
사랑해요
#selfie #movie #instadaily #instadaily #instagood #ootd #ootd #dailylook #데
----------------------------------------------------------------------

【 생성 2 】 시작 단어: 함께
함께 할 수 있는 교육 프로그램이 더 많아졌으면 좋겠다.
지금까지 학교 현장에서 교육 받은 것을 바탕으로 학교 현장의 의견을 수렴하고 이를 정책에 반영하고 싶다.
또한 교육청에서도 학교 현장에서 다양한 의견을 수렴하고 반영하기 위해 노력하고 있다.
이러한
----------------------------------------------------------------------

【 생성 3 】 시작 단어: 꿈을
꿈을 위해선 안간힘을 써주는 사람만이 가능하다"고 강조했다.
이어 "아무도 없는 곳에서, 아무도 없는 곳에서, 그리고 모두가 행복한 꿈을 꾸고 있는 사람은 누군가보다 더 행복할 수 있다는 것"이라며 "무엇보다 사람이
----------------------------------------------------------------------

💡 실제 파인튜닝:
   1. 더 많은 K-pop 가사 데이터 수집 (1000+ 곡)
   2. 위 TrainingArguments로 학습 실행
   3. 파인튜닝된 모델로 더 나은 결과 생성
   4. Hugging Face Hub에 업로드하여 공유




---



## **K-pop 가사 데이터셋을 이용한 미세튜닝**

### **1단계: 깃허브에서 K-pop 가사 데이터셋 다운로드 및 전처리**

In [3]:
# 기존에 있던 폴더강제  삭제하기
!rm -rf /content/Kpop-lyric-datasets

In [4]:
# 1. 깃허브 저장소 복제 (데이터 다운로드)
!git clone https://github.com/EX3exp/Kpop-lyric-datasets.git

# 2. 필요한 라이브러리 설치
# datasets: 허깅페이스의 데이터셋 라이브러리
# transformers: 모델 및 토크나이저 라이브러리
# accelerate: 학습 가속화
!pip install datasets transformers accelerate

Cloning into 'Kpop-lyric-datasets'...
remote: Enumerating objects: 26227, done.
remote: Counting objects: 100% (26227/26227), done.
remote: Compressing objects: 100% (25585/25585), done.
remote: Total 26227 (delta 656), reused 26196 (delta 639), pack-reused 0 (from 0)
Receiving objects: 100% (26227/26227), 24.97 MiB | 22.47 MiB/s, done.
Resolving deltas: 100% (656/656), done.
Updating files: 100% (25885/25885), done.


- **(필터링 추가) 2020년 이후 데이터셋만 추출하기**
    - 전체 데이터셋으로 하면 시간이 오래 걸린다.(A100 40분 이상)

In [5]:
import os
import json # .json 파일을 파싱(parsing)하기 위해 내장 라이브러리 import
from glob import glob
import re   # 정규표현식 라이브러리 추가

# 3. 정확한 JSON 파일 경로 탐색
data_path = "Kpop-lyric-datasets/melon/monthly-chart"
search_pattern = os.path.join(data_path, "*", "*", "*.json")
all_json_files = glob(search_pattern)

print(f"발견된 총 JSON 파일 개수: {len(all_json_files)}개")

if not all_json_files:
    print("!!! 오류: JSON 파일을 찾지 못했습니다. git clone이 성공했는지, 경로가 올바른지 확인하세요.")


# 4. JSON 파일을 '단일 딕셔너리'로 간주하고 가사 추출
train_file_path = "kpop_lyrics_train.txt"
total_lyrics_count = 0
processed_files_count = 0 # (추가) 2020년 이후 처리된 파일 카운트

# 파일 경로에서 연도를 추출하기 위한 정규표현식 컴파일
# 'melon-' 뒤에 나오는 4자리 숫자(\d{4})를 찾습니다.
year_pattern = re.compile(r'melon-(\d{4})')

with open(train_file_path, "w", encoding="utf-8") as train_file:
    for file_path in all_json_files: # 각 JSON 파일을 순회

        # --- 연도 필터링 로직 ---
        year_match = year_pattern.search(file_path)

        if not year_match:
            # 'melon-YYYY' 패턴이 파일 경로에 없으면 건너뜁니다.
            continue

        # 정규표현식으로 찾은 연도(문자열)를 정수(int)로 변환
        year = int(year_match.group(1))

        # 2020년 미만 데이터는 건너뜁니다.
        if year < 2020:
            continue
        # --------------------------------

        # (2020년 이상인 파일만 처리)
        processed_files_count += 1

        try:
            with open(file_path, "r", encoding="utf-8") as f:
                # JSON 파일 1개를 딕셔너리(song_data)로 바로 로드합니다.
                song_data = json.load(f)

                # 1. 'lyrics' 객체를 song_data에서 직접 가져옵니다.
                lyrics_object = song_data.get("lyrics")

                # 2. lyrics_object가 존재하고, 딕셔너리 형태일 경우에만 'lines' 리스트를 찾습니다.
                if lyrics_object and isinstance(lyrics_object, dict):
                    lines_list = lyrics_object.get("lines")

                    # 3. lines_list가 존재하고, 리스트 형태일 경우에만 가사를 합칩니다.
                    if lines_list and isinstance(lines_list, list):
                        full_lyrics = "\n".join(lines_list).strip()

                        if full_lyrics:
                            train_file.write(full_lyrics + "\n\n<|endoftext|>\n\n")
                            total_lyrics_count += 1

        except json.JSONDecodeError:
            print(f"JSON 파싱 오류: {file_path} (파일이 손상되었을 수 있습니다)")
        except AttributeError as e: # 'str' object has no attribute 'get' 오류 발생 시
             print(f"!!! 데이터 구조 오류 ({file_path}): {e}. JSON 파일이 예상과 다른 구조일 수 있습니다.")
        except Exception as e:
            print(f"파일 처리 중 알 수 없는 오류 ({file_path}): {e}")

print(f"'{train_file_path}' 파일 생성 완료!")
print(f"--- (필터링 적용) 2020년 이후 데이터만 처리 ---")
print(f"처리된 파일 개수: {processed_files_count}개 (전체: {len(all_json_files)}개 중)")
print(f"총 {total_lyrics_count}개의 가사를 성공적으로 추출하여 파일에 저장했습니다.")


발견된 총 JSON 파일 개수: 25876개
'kpop_lyrics_train.txt' 파일 생성 완료!
--- (필터링 적용) 2020년 이후 데이터만 처리 ---
처리된 파일 개수: 4500개 (전체: 25876개 중)
총 4490개의 가사를 성공적으로 추출하여 파일에 저장했습니다.


### **2단계: K-pop 가사 데이터셋으로 KoGPT2 모델 파인튜닝**

In [6]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TextDataset,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)

# 1. 파인튜닝할 기반 모델 및 토크나이저 선택
BASE_MODEL = "skt/KoGPT2-base-v2"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL,
    bos_token='<s>', # 문장 시작
    eos_token='</s>', # 문장 끝
    unk_token='<unk>',
    pad_token='<pad>',
    mask_token='<mask>')

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL)
model.resize_token_embeddings(len(tokenizer)) # 토크나이저에 맞게 모델 임베딩 크기 조절

# 2. 학습 데이터셋 로드
# 우리가 1단계에서 만든 kpop_lyrics_train.txt 파일을 사용합니다.
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_file_path,  # kpop_lyrics_train.txt
    block_size=128              # 메모리에 맞게 블록 크기 조절 (128~256)
)

# 3. 데이터 콜레이터 설정 (배치 생성)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Masked Language Model (BERT)가 아닌 Causal LM (GPT) 방식
)

# 4. 학습 설정 (TrainingArguments)
# 파인튜닝된 모델이 저장될 경로
output_dir = "./kpop-kogpt2-finetuned"

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=3,     # 에포크 수 (3~5 정도가 적당)
    per_device_train_batch_size=4,  # Colab T4 메모리에 맞게 배치 크기 (4 또는 8)
    save_steps=10_000,      # (예제라 저장 스텝은 크게 잡음)
    save_total_limit=2,
    logging_steps=100,      # 100 스텝마다 로그 출력
    fp16=True, # T4 GPU에서 학습 속도 향상을 위해 16비트 사용
)

# 5. 트레이너(Trainer) 생성 및 학습 시작
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

print("=== K-pop 가사 데이터로 모델 파인튜닝을 시작합니다 ===")
trainer.train()
print("=== 파인튜닝 완료! ===")

# 6. 학습된 모델 저장
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"파인튜닝된 K-pop 모델이 '{output_dir}'에 저장되었습니다.")

# 파인튜닝된 K-pop 모델이 './kpop-kogpt2-finetuned'에 저장되었습니다.

config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


=== K-pop 가사 데이터로 모델 파인튜닝을 시작합니다 ===


...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jayun7673 (jayun7673-kt-techup) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,3.462000
200,3.251800
300,3.059600
400,2.892100
500,2.768000
600,2.751600
700,2.584700
800,2.596500
900,2.515200
1000,2.399700


=== 파인튜닝 완료! ===
파인튜닝된 K-pop 모델이 './kpop-kogpt2-finetuned'에 저장되었습니다.


### **3단계: 파인튜닝된 모델로 K-pop 가사 생성하기**

In [7]:
from transformers import pipeline

# 1. 우리가 2단계에서 파인튜닝한 모델 로드
# BASE_MODEL이 아닌, 우리가 저장한 output_dir 경로를 사용합니다.
finetuned_model_path = "./kpop-kogpt2-finetuned"

# 'text-generation' 파이프라인을 사용하면 쉽게 텍스트를 생성할 수 있습니다.
# device=0은 GPU를 사용하겠다는 의미입니다. (CPU는 -1)
generator = pipeline(
    'text-generation',
    model=finetuned_model_path,
    tokenizer=finetuned_model_path,
    device=0 if torch.cuda.is_available() else -1
)

# 2. 가사 생성 테스트
# 생성할 가사의 시작 프롬프트(Seed text)를 제공합니다.
# 이 프롬프트에 이어지는 가사를 모델이 생성합니다.
seed_text = "차가운 바람이 불어와"
# seed_text = "오늘 밤 너와 나"
# seed_text = "이 비트가 날 미치게 해"
# seed_text = "저 우주 끝까지 하이퍼 스페이스"
# seed_text = "네가 만든 쿠키 맛이 어때"
# seed_text = "Oh baby, You're so fine"
# seed_text = "Oh baby, 넌 너무 멋져"


# 3. 가사 생성 옵션 설정
# num_return_sequences: 몇 개의 다른 버전 가사를 생성할지
# max_length: 생성될 가사의 최대 길이 (프롬프트 포함)
# do_sample=True: 다양한 가사를 생성 (True 권장)
# top_k, top_p: 생성될 단어의 다양성 조절 (k=50, p=0.95가 일반적)
# no_repeat_ngram_size: 특정 n-gram(예: 2단어)이 반복되지 않도록 하여 "사랑해 사랑해 사랑해" 같은 반복 방지
generated_lyrics = generator(
    seed_text,
    num_return_sequences=3, # 3가지 버전의 가사 생성
    max_length=100,         # 최대 100 토큰 길이
    do_sample=True,
    top_k=50,
    top_p=0.95,
    no_repeat_ngram_size=2,
    early_stopping=True
)

print(f"--- 프롬프트: '{seed_text}' ---")
for i, result in enumerate(generated_lyrics):
    print(f"\n[생성 가사 {i+1}]")
    print(result['generated_text'])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


--- 프롬프트: '차가운 바람이 불어와' ---

[생성 가사 1]
차가운 바람이 불어와
눈을 감을 때마다
향기로운 네 맘이
내게 전해지네
너는 무슨 생각해
나란히 누워서
저 별을 바라볼 때면
나와 같은 마음인지는 몰라도
너와 함께 있는
이 순간이 난 소중해
나의 모든 걸 주고 싶어
너의 모든 순간을 난 너에게 줄게
세월이 흘러가도 난 변하지 않아
오직 너 아니면 안 된다고
외치고 싶어 그저
그저 내 곁에만 있어줘
떠나지 말아줘요
참 많이 어색했었죠 널 처음 만난 날
멀리서 좋아하다가 들킨 사람처럼
숨이 가득 차올라서
아무 말 하지 못했는데
왜 난 늘 뒤돌아보면
혼자 있으면 나도 그래
늘 혼자 센척했지만 많이 두려워해 너
사실 난 네게 부족하지만 참 많이 부족해 많은 걸
사랑을 많이 했죠
하지만 많이 부족한 나를
사람에게
누군가 있어서 난 행복해 줘야만 해서
이젠 내겐 소중한 사람
널 만날 수 있어
있다는 걸 잘 알아
영원하는 걸 알면서도 그댈 너무 사랑해 난 아직도 믿어
행복했어
오늘도약은 항상 그랬어 헤어지기 싫어
난 언제나
돌아와

[생성 가사 2]
차가운 바람이 불어와
두 볼을 스칠 때 (Hey)
또 괜히 나 왜 두근거려 왜 (왜)

Call me "Desperado"
뭐래도 몰라도 돼
오늘만은 내 멋대로
Go far away

치맛바람이 불거든
내게도 사랑이 찾아와 Yeah
뜨거운 햇살이 비춰와
아름다운 여름밤이라구요

살랑 살랑 불어 기분 좋은 날에
바람 바람 바람 이 시원한 바람
찰랑 찰랑 이 푸른 바닷가에
너와 함께 있는 이 순간이 난 소중해
나랑 같이 걸을래
혹시 내일은 뭐해 날씨도 좋은 날이야
널 위해 준비된
선물 같아 널 안으면
잠들지 않는 바다 위를
와이어링하는 재떨이
하늘 아래 너와 나랑
함께 걸은 지금 더 같이 걸어볼래 내일의 미래
은은모래 위
파도 위를 난 climple my city
더 파도에 더 크게 외쳐 더 파도와 소리쳐 또 파도를 가
저기 날아 fly floor everyday tour
작은 달아
어둠 사이로 더 자유롭게 날

### **[실습] 다양한 제시문장으로 가사 생성 테스트하기**

코드의 #2. 가사 생성 테스트 부분에서 제시한 **제시문장**을 다양하게 입력해보고 가사 생성 결과를 확인해 보세요.



---



## **[미션] 데이터셋을 수집하고 전처리하여 모델을 2차 튜닝하기**
앞에서 튜닝된 모델에 새롭게 수집된 데이터셋을 넣어서 2차 튜닝 모델을 만들어 보세요.



---

